# Access Azure resources from an online endpoint with a user-assigned managed identity

Additional packages are required for this example:

* Microsoft Azure Msi Management Client 
* Microsoft Azure Storage Client
* Microsoft Azure Authorization Management Client

Install them with the following code:

In [ ]:
%pip install --pre azure-mgmt-msi
%pip install --pre azure-mgmt-storage
%pip install --pre azure-mgmt-authorization

## 1. Configure variables

### 1.1 Assign variables for the workspace and deployment

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

endpoint_name = "<ENDPOINT_NAME>"

In [1]:
# TODO: Remove
subscription_id = "6fe1c377-b645-4e8e-b588-52e57cc856b2"
resource_group = "role-creation-sample"
workspace_name = "test-workspace"

endpoint_name = "roletest7"

### 1.2 Specify the storage details and file to access
A text file is located in the `managed-identities` folder called `hello.txt`. 

In [ ]:
storage_account_name = "<STORAGE_ACCOUNT_NAME>"
storage_container_name = "<CONTAINER_TO_ACCESS>"
file_name = "<FILE_TO_ACCESS>"

In [2]:
# TODO: Remove

storage_account_name = "rolestestuai3"
storage_container_name = "blob4"
file_name = "hello.txt"

### 1.3 Decide on the name of your user identity:

In [ ]:
uai_name = "<USER_ASSIGNED_IDENTITY_NAME>"

In [3]:
uai_name = "testidentity5"

### 1.4 Retrieve the workspace location:

In [4]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    ManagedOnlineEndpoint,
    Model,
    CodeConfiguration,
    Environment,
    IdentityConfiguration,
    UserAssignedIdentity
)

credential = AzureCliCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

workspace_location = ml_client.workspaces.get(workspace_name).location

## 2. Configure deployment

### 2.1 Define an endpoint configuration

### 2.2 Define a deployment configuration

In [5]:
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=Model(path="../../model-1/model/"),
    code_configuration=CodeConfiguration(
        code="../../model-1/onlinescoring/", scoring_script="score_managedidentity.py"
    ),
    environment=Environment(
        conda_file="../../model-1/environment/conda.yml",
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1",
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
    environment_variables={
        "STORAGE_ACCOUNT_NAME": storage_account_name,
        "STORAGE_CONTAINER_NAME": storage_container_name,
        "FILE_NAME": file_name,
        # We will update this after creating an identity
        "UAI_CLIENT_ID": "uai_client_id_place_holder",
    },
)

## 3. Create the managed identity

### 3.1 Get a handle to the `ManagedServiceIdentityClient`

In [6]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.msi import ManagedServiceIdentityClient
from azure.mgmt.msi.models import Identity

credential = AzureCliCredential()
msi_client = ManagedServiceIdentityClient(
    subscription_id=subscription_id,
    credential=credential,
)

### 3.2 Create the identity

In [7]:
msi_client.user_assigned_identities.create_or_update(
    resource_group_name=resource_group,
    resource_name=uai_name,
    parameters=Identity(location=workspace_location),
)

### 3.3 Retrieve the identity object

In [8]:
uai_identity = msi_client.user_assigned_identities.get(
    resource_group_name=resource_group,
    resource_name=uai_name,
)
uai_identity.as_dict()

{'id': '/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/resourcegroups/role-creation-sample/providers/Microsoft.ManagedIdentity/userAssignedIdentities/testidentity5',
 'name': 'testidentity5',
 'type': 'Microsoft.ManagedIdentity/userAssignedIdentities',
 'tags': {},
 'location': 'eastus2',
 'tenant_id': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'principal_id': 'a52aa6e7-b53a-4e34-96d4-3cce696c585c',
 'client_id': '7c5ed570-2b60-49ff-8253-abe2877bf4ec'}

## 4. Create storage account and container

### 4.1 Get a handle to the `StorageManagementclient`

In [9]:
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.mgmt.storage import StorageManagementClient
from azure.storage.blob import ContainerClient
from azure.mgmt.storage.models import Sku, StorageAccountCreateParameters, BlobContainer

credential = AzureCliCredential()
storage_client = StorageManagementClient(
    credential=credential, subscription_id=subscription_id
)

### 4.2 Define and create the storage account

In [10]:
storage_account_parameters = StorageAccountCreateParameters(
    sku=Sku(name="Standard_LRS"), kind="Storage", location=workspace_location
)

poller = storage_client.storage_accounts.begin_create(
    resource_group_name=resource_group,
    account_name=storage_account_name,
    parameters=storage_account_parameters,
)

poller.wait()

storage_account = poller.result()

### 4.3 Create a blob container

In [11]:
blob_container = storage_client.blob_containers.create(
    resource_group_name=resource_group,
    account_name=storage_account_name,
    container_name=storage_container_name,
    blob_container=BlobContainer(),
)

### 4.4 Get the storage account key and create a `ContainerClient`

In [12]:
res = storage_client.storage_accounts.list_keys(
    resource_group_name=resource_group,
    account_name=storage_account_name,
)
key = res.keys[0].value

container_client = ContainerClient(
    account_url=storage_account.primary_endpoints.blob,
    container_name=storage_container_name,
    credential=key,
)

### 4.5 Upload a blob to the container

In [13]:
file_path = "hello.txt"
with open(file_path, "rb") as f:
    container_client.upload_blob(name=file_name, data=f.read())

ResourceExistsError: The specified blob already exists.
RequestId:77f2f081-c01e-00dd-59f8-c91d72000000
Time:2022-09-16T18:19:49.6922690Z
ErrorCode:BlobAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobAlreadyExists</Code><Message>The specified blob already exists.
RequestId:77f2f081-c01e-00dd-59f8-c91d72000000
Time:2022-09-16T18:19:49.6922690Z</Message></Error>

## 5. Create an online endpoint

### 5.1 Create the endpoint

In [24]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key", identity=IdentityConfiguration(
    type="UserAssigned", 
    user_assigned_identities=
    [
        {"resource_id": uai_identity.id}
    ])
)


#UserAssignedIdentity(
   # UserAssignedIdentity(resource_id=uai_identity.id)
           # resource_id=uai_identity.id)
#endpoint.identity = endpoint.identity.from_dict(
    #{"type": "UserAssigned", "user_assigned_identities": {uai_identity.id: {}}},
#)

ml_client.online_endpoints.begin_create_or_update(endpoint) # Create endpoint with user assigned identity

TypeError: 'UserAssignedIdentity' object is not subscriptable

### 5.2 Update its identity

In [48]:
#endpoint = ml_client.online_endpoints.get(endpoint_name)
#endpoint.identity = endpoint.identity.from_dict(
#    {"type": "UserAssigned", "user_assigned_identities": {uai_identity.id: {}}}
#)

#ml_client.online_endpoints.begin_create_or_update(endpoint)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://roletest5.eastus2.inference.ml.azure.com/score', 'swagger_uri': 'https://roletest5.eastus2.inference.ml.azure.com/swagger.json', 'name': 'roletest5', 'description': None, 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/resourcegroups/role-creation-sample/providers/microsoft.machinelearningservices/workspaces/test-workspace/onlineendpoints/roletest5', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oe:5d0ba9bd-465c-466f-a133-462c9bfa6515:3d9be8ec-bc96-429e-9ae1-bca937d24f60?api-version=2022-02-01-preview'}, 'id': '/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/resourceGroups/role-creation-sample/providers/Microsoft.MachineLearningServices/workspaces/test-workspace/onli

### 5.3 Confirm identity details

In [49]:
endpoint = ml_client.online_endpoints.get(endpoint_name)
endpoint.identity.as_dict()

{'principal_id': '00000000-0000-0000-0000-000000000000',
 'tenant_id': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'type': 'UserAssigned',
 'user_assigned_identities': {'/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/resourcegroups/role-creation-sample/providers/Microsoft.ManagedIdentity/userAssignedIdentities/testidentity2': {'principal_id': 'b1a913f4-88fb-4605-9347-71b4013a19b7',
   'client_id': '8e85c41b-0093-4454-9ae9-ba43ad584cba'}}}

## 6. Give access permission to the managed identity

### 6.1 Get an `AuthorizationManagementClient` to list Role Definitions

In [50]:
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.authorization.v2018_01_01_preview.models import RoleDefinition
import uuid

role_definition_client = AuthorizationManagementClient(
    credential=credential,
    subscription_id=subscription_id,
    api_version="2018-01-01-preview",
)

### 6.2 Get an `AuthorizationManagementClient` to make Role Assignments

In [51]:
from azure.mgmt.authorization.v2020_10_01_preview.models import (
    RoleAssignment,
    RoleAssignmentCreateParameters,
)

role_assignment_client = AuthorizationManagementClient(
    credential=credential,
    subscription_id=subscription_id,
    api_version="2020-10-01-preview",
)

### 6.2 Get User-assigned identity details

In [52]:
uai_identity = msi_client.user_assigned_identities.get(
    resource_group_name=resource_group, resource_name=uai_name
)
uai_principal_id = uai_identity.principal_id
uai_client_id = uai_identity.client_id

### 6.3 Give permission to the user storage account

In [53]:
role_name = "Storage Blob Data Reader"
scope = storage_account.id

role_defs = role_definition_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

role_assignment_client.role_assignments.create(
    scope=scope,
    role_assignment_name=str(uuid.uuid4()),
    parameters=RoleAssignmentCreateParameters(
        role_definition_id=role_def.id, principal_id=uai_principal_id, principal_type="ServicePrincipal"
    ),
)

### 6.4 Retrieve the workspace and container registry objects

In [54]:
workspace = ml_client.workspaces.get(workspace_name)
container_registry = workspace.container_registry

### 6.5 Give permission to the container registry

In [55]:
role_name = "AcrPull"
scope = container_registry

role_defs = role_definition_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

role_assignment_client.role_assignments.create(
    scope=scope,
    role_assignment_name=str(uuid.uuid4()),
    parameters=RoleAssignmentCreateParameters(
        role_definition_id=role_def.id, principal_id=uai_principal_id, principal_type="ServicePrincipal"
    ),
)

### 6.6  Give permission to the workspace storage account

In [56]:
role_name = "Storage Blob Data Reader"
scope = workspace.storage_account

role_defs = role_definition_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

role_assignment_client.role_assignments.create(
    scope=scope,
    role_assignment_name=str(uuid.uuid4()),
    parameters=RoleAssignmentCreateParameters(
        role_definition_id=role_def.id, principal_id=uai_principal_id, principal_type="ServicePrincipal"
    ),
)

## 7. Create a deployment with your configuration

### 7.1 Update the deployment configuration with the UAI Client ID

In [57]:
deployment.environment_variables["UAI_CLIENT_ID"] = uai_client_id # Is this correct?

### 7.2 Create the deployment

In [58]:
ml_client.online_deployments.begin_create_or_update(deployment)

Check: endpoint roletest5 exists
Creating/updating online deployment blue 

.........................................................................................

HttpResponseError: (None) ResourceNotReady: User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready
Code: None
Message: ResourceNotReady: User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready
Exception Details:	(None) ResourceNotReady: User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready
	The build log is available in the workspace blob store "testworkspace5376630068" under the path "/azureml/ImageLogs/2b684463-74b2-466e-afbd-a86cea622144/build.log"
	Code: None
	Message: ResourceNotReady: User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready
	The build log is available in the workspace blob store "testworkspace5376630068" under the path "/azureml/ImageLogs/2b68

### 7.3 Check the status of the deployment

In [ ]:
deployment = ml_client.online_deployments.get(
    endpoint_name=endpoint_name, name=deployment.name
)
print(deployment)

### 7.4 Get the deployment logs

In [ ]:
ml_client.online_deployments.get_logs(deployment.name, deployment.endpoint_name, 1000)

### 7.5 Set traffic to 100% for deployment

In [ ]:
endpoint.traffic = {str(deployment.name): 100}
ml_client.begin_create_or_update(endpoint)

## 8. Confirm your endpoint deployed successfully

In [ ]:
sample_data = "../../model-1/sample-request.json"
ml_client.online_endpoints.invoke(endpoint_name=endpoint_name, request_file=sample_data)

## 9. Delete the endpoint and storage account

### 9.1 Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(endpoint_name)

### 9.2 Delete the storage account

In [ ]:
storage_client.storage_accounts.delete(
    resource_group_name=resource_group, account_name=storage_account_name
)